# Численные методы. Проектная работа. 
# Определение стоимости автомобилей

## Описание проекта.

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:

* качество предсказания
* скорость предсказания
* время обучения

## Подготовка данных

Импротируем необходимые библиотети и загрузим данные

In [ ]:
import pandas as pd
import numpy as np
import re
import datetime



import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor

from catboost import Pool, CatBoostClassifier, CatBoostRegressor


import lightgbm as lgb

from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, OrdinalEncoder
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

import warnings
warnings.filterwarnings('ignore')

In [ ]:
random_magic = 1024

In [ ]:
try:
    data = pd.read_csv('autos.csv')
except:
    data = pd.read_csv('/datasets/autos.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

Набор данных предстален 16 столбцами и 354369 строками.

Признаки:

* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* NotRepaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

Целевой признак:

* Price — цена (евро)

В таблице 9 строковых и 7 целочисленных атрибутов, нужно преобразовать столбцы с датами в тип данных даты.

Имеются пропуски в столбцах VehicleType, Gearbox, Model, FuelType, NotRepaired.

Необходимо приведение названий признаков к "змеиному регистру".

Кроме того, в отдельных атрибутах явно имеются аномальные значения:

* '9999' - год регистрации
* 0 - месяц регистрации
* 0 - цена автомобиля
* 20000 - мощность (л. с.)

Решение о способе вставки пропусков и замены указанных аномальных значений целесообразно принять на этапе последующего изучения и предобработки данных.

Данные объявлений также содержат персональную информацию пользователей - почтовый код и время последней активности (видимо, на ресурсе, где представлены объявления), которые целесообразно сразу исключить из рассмотрения.

Помимо прочего в данных содержатся признаки, описывающие само объявление, а не продаваемый автомобиль - количество фотографий, дата создания и выгрузки объявления. При этом признак количества фотографий содержит только одно значение - 0. Этот признак также целесообразно сразу исключить из рассмотрения.

## Подготовка данных

Приведем названия признаков к " змеиному регистру"

In [ ]:
data.rename(columns=lambda x: re.sub('(?!^)([A-Z]+)', r'_\1',x).lower(), inplace=True)
data.head()

Выполним преобразование типа данных для дат.

In [ ]:
data['date_crawled'] = pd.to_datetime(data['date_crawled'])
data['date_created'] = pd.to_datetime(data['date_created'])

Проверим данные на дубликаты

In [ ]:
duplicates = data[data.duplicated()] 
duplicates

Удалим их.

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

Рассморим один интересный момент. Признак "была машина в ремонте или нет" допускает разное толкование. Если мы будем опираться на английское название столбца 'not repaired', то оно будет скорее трактоваться как "не отремотирована" - то есть ответ 'yes' говорит, что она находится в битом состоянии, а ответ 'no' - отремонтирована, то есть на ходу.

Если же мы обратимся к русской трактовке признака, которая дана в описании данных, то на вопрос "была машина в ремонте или нет" положительный ответ "да" будет означать, что она отремонтирована. Для прояснения ситуации отдельно исследуем этот признак.

In [ ]:
sns.set_style('whitegrid')
sns.boxplot(data = data, x='not_repaired', y='price', palette='Set2', fliersize=0.5)
plt.title('Цена на автомобили в разрезе "not_repaired"');

In [ ]:
data['not_repaired'].value_counts()

В объявлениях, где в 'not_repaired' указан ответ 'no' - медианная цена гораздо выше, чем там, где 'yes'. Значит, мы для себя можем определить, что этот признак выражает на ходу машина или нет.

'not_repaired' - 'не отремонтирована': 'no' - нет, не "не отремонтирована" - значит, на ходу; 'yes' - да, "не отремонтирована", значит, не на ходу.

Также анализ объявления реальных площадок показал, что есть такой параметр как "битая" - и там объявления машин после аварий. Значит, признак 'not_repaired' для нас выступает в роли такого флажка - 'no' - все в порядке, можно ездить, 'yes' - битая, ездить вряд ли получится.

Если бы это был реальная задача, ты мы бы сделали запрос в источнику выгрузки базы данных.

Рассмотрим остальные признаки поподробнее

In [ ]:
data.sort_values(by='date_crawled', ascending=True)

Скачанные из базы анкеты охватывают временной промежуток около месяца - примерно март 2016 года.

In [ ]:
data.sort_values(by='date_created', ascending=True)

При этом анкеты создавались в течение двух лет - с марта 2014 года.

In [ ]:
data.sort_values(by='last_seen', ascending=True)

Пользователи также появлялись на сайте в течение последнего месяца, значит, можно говорить о том, что ценовые данные даже для старых объявлений - актуальны.

Удалим признаки "количество фотографий автомобиля", "почтовый код" и "время последней активности".

In [ ]:
data.drop(columns=['number_of_pictures', 'postal_code', 'last_seen'], inplace=True)

Чтобы оценить, адекватные ли в нашей базе данные, построим ящики с усами

In [ ]:
features_to_plot = {'price':0, 'registration_year':1, 'power':2,'kilometer':3}
fig, axes = plt.subplots(1,4, figsize = (12,6))
for feature, pos in features_to_plot.items():
    sns.boxplot(x=data[feature], orient='h', palette='Set2', fliersize=0.5, ax=axes[pos]);

'price' - медиана по цене составляет около 2700 тыс. евро, видимо, в основном представлены бюджетные автомобили.
Есть автомобили с ценой 0 - нам предстоит понять, что это - неявные пропуски или продавцы отдают в дар.

'regisrtation_year' - много значений выше 2500, похоже, на ошибки, нужно будет посмотреть на них и, скорее всего, избавиться.

'power' - также неадекватно высокая мощность, возможно, предстоит избавиться.

'kilometer' - столбец с пробегом не вызывает подозрений, кроме того, что медиана - 150 тыс. км и максимальное значение такое же. Возможно, при выгрузке данных стоял какой-то фильтр, например, пробег не более 150 000 км, поэтому будем работать с тем, что есть.

Посмотрим на обьявления с неадекватно низкой ценой, менее 100 евро

In [ ]:
df = data.query('price < 100')
df['price'].value_counts()

10772 обьявления, возможно, это автомобили не на ходу, посмотрим имел ли место ремонт.

In [ ]:
low_price = data.query('price < 100')
low_price.info()

In [ ]:
low_price['not_repaired'].value_counts()

Исследования показывают, что 3955 автомобилей с очень низкой ценой были отремонтированы, 2005 - нет и остальная часть объявлений с низкой ценой без данных о ремонте. То есть, если бы мы однозначно увидели, что все автомобили с низкой ценой в столбце 'not_repaired' имеют 'yes' - то есть они были бы не находу - тогда мы теоретически могли бы сказать, что в этом срезе у нас автохлам. С учетом того, что это целевой признак, проявлять фантазию и пытаться заполнить модами или медианами выглядит опасным, поэтому придется избавиться от этих данных.

In [ ]:
data = data.query('price >= 100')

Разберем подробно столбец с пропусками о ремонте, попробуем воссстановить данные.
Отберем объявления, где указано, что машина в ремонте не была.

In [ ]:
data[data['not_repaired'] == 'no'].describe(include='all')

Средняя стоимость этих авто 4382 евро, медианная - 3799. 

Посмотрим на те же метрики у объявлений, где указано, что авто было в ремонте.

In [ ]:
data[data['not_repaired'] == 'yes'].describe(include='all')

Средняя стимость этих автомобилей 2061 евро, медиана - 1000.

Теперь посмотрим на объявления, где наличие или отсутствие ремонта не указано.

In [ ]:
data[data['not_repaired'].isna()].describe(include='all')

Здесь средняя стоимость 2903 евро и медиана 1600. Можно было бы сделать выбор в пользу заполнения параметром "yes", так как цены больше тяготеют к выборке автомобилей, бывших в ремонте, но всё же не будем рисковать,ситуация, когда факт проведения ремонта не указан, все-таки отличается от ситуации, когда указано, что автомобиль не ремонтировался. Но и удалять такой массив данных не хотелось бы, поэтому принято решение вместо пропусков создать отдельную категорию - unknown.

In [ ]:
data['not_repaired'] = data['not_repaired'].fillna('unknown')

Далее проанализируем признак 'год регистрации'. В наших данных не может быть автомобилей с годом регистрации больше 2016, была версия, что данные пополнялись но она разбилась о дату регистрации анкеты, объявление не могло быть опубликовано ранее, чем автомобиль был зарегистрирован.

In [ ]:
data[data['registration_year'] >= 2017].describe(include='all')

In [ ]:
display(data[data['registration_year']  >= 2017])

Информация о данных этого среза и визуальный осмотр объявлений говорит нам о том, что здесь не наблюдается определенной системы в пропусках - представлены машины различных брендов и моделей, типов топлива. При этом почти у всех у них есть пропуски в типе кузова. Почему эти машины попали в нашу выборку из будущего выяснить не представляется возможным, удалим эти данные.

In [ ]:
data = data.query('registration_year <= 2017')

Посчитаем долю объявлений по очень старым автомобилям.

In [ ]:
display('Доля объявлений с указанием года регистрации до 1980: {0:.2%}'
      .format(data.query('registration_year < 1980')['registration_year'].count() / data.shape[0]))

Без сожалений избавляемся и от них.

Кроме того, удалим данные имеющие большое количество пропусков/аномалий. Значение марки автомобиля sonstige_autos с немецкого - прочие автомобили, что никак не добавляет ясности в наши данные.

In [ ]:
data.drop(
    data.query('vehicle_type.isnull() and gearbox.isnull() and power == 0 \
              and fuel_type.isnull() and brand==\'sonstige_autos\' and not_repaired.isnull()', engine='python').index,
    inplace=True
)

In [ ]:
data = data.query('registration_year >= 1980')

In [ ]:
data['registration_month'].value_counts()

Объявлений с 0 в качестве месяца регистрации порядка 10% от общего числа объявлений. С учетом того, что данные по месяцу регистрации распределены достаточно равномерно, то и замену значения 0 целесообразно сделать таким образом, чтобы не изменить исходное распределение.

Сделаем замену случайным образом на число из диапазона (1, 12) - для этого индекс записи приведем к модулю 12 и добавим 1.

In [ ]:
data.loc[data['registration_month'] == 0, 'registration_month'] = (
    data.query('registration_month == 0').index % 12 + 1
)

In [ ]:
data['registration_month'].value_counts()

Выше было отмечено, что значение sonstige_autos является аномальным (соответствует категории прочие марки автомобилей). Очевидно, что марка автомобиля является одним из ключевых признаков - определяет модель, а также неявно определяет многие прочие характеристики (например, в заданный период производитель может выпускать только автомобили определенного типа кузова или определенной мощности). Необходимо оценить долю автомобилей без указания марки в общем объеме объявлений.

In [ ]:
data.query('brand == \'sonstige_autos\'')

In [ ]:
data.query('brand==\'sonstige_autos\' and (vehicle_type.isnull() or gearbox.isnull() or power == 0 \
              or fuel_type.isnull() or not_repaired.isnull() or price in (0, 1))', engine='python')

Доля автомобилей без указания марки составляет всего 0.6% (ближе к нижней части списка частот встречаемости). Кроме того, среди указанных автомобилей есть те, для которых не указаны и прочие параметры - мощность, тип кузова, тип коробки передач. 

Целесообразно удалить все данные об автомобилях без указания марки, поскольку их доля в общем объеме мала и кроме того среди этих данных много пропусков/аномалий в прочих атрибутах.

In [ ]:
data = data.query('brand != \'sonstige_autos\'')

Для последующего стратифицированного разделения данных, целесообразно добавить дополнительный атрибут, позволяющий отделить модели, занимающие менее 1% рынка от других.

In [ ]:
top_models = [
    'volkswagen',        
    'opel',
    'bmw',
    'mercedes_benz', 
    'audi',
    'ford',
    'renault',
    'peugeot',
    'fiat',
    'seat',
    'mazda',
    'skoda',
    'smart',
    'citroen',
    'nissan',
    'toyota',
    'hyundai',
]

In [ ]:
data['brand_type'] = data.apply(
    lambda row: row['brand'] if row['brand'] in top_models else 'other',    
    axis=1
)

Оценим значения мощностей двигателей в представленном наборе данных.
Сразу отбросим объявление с указанной мощностью более 1000 л.с.

Можно было бы сделать замену - разделить указанное значение на 20 (некий средний переводной коэффициент из объема в мощность, но это было бы искажение данных, так как доподлинно нам не известно, что это указание объема двигателя. 

In [ ]:
data =data[data['power'] < 1000]

In [ ]:
plt.boxplot(x=data['power']);

Посмотрим на нулевые значения.

In [ ]:
data[data['power'] < 1]

Вероятно объявления, в которых значение мощности просто не было указано, их доля относительно всего объема - порядка 10%.

Предлагается нулевые значения заменить на основе средних значений по производителям и годам производства. Чтобы не переобучить модель указанную операцию целесообразно выполнить после разделения всей выборки на обучающую и тестовую (на обучающей собрать статистику и использовать ее для замены значений на обучающей и тестовой выборках).

Значения в интервале от 1 до 45 целесообразно удалить как выбросы.

Зона выбросов - значения свыше 300 л.с. также присутствует, но исключение этих данных нецелесообразно - в них присутствуют как ошибки (например mitsubishi colt мощностью 953 л.с.), так и верные данные (porsche cayenne 450 л.с.).

In [ ]:
data = data.query('power == 0 or power >= 45')

Оценим распределение значений пробега.

In [ ]:
data['kilometer'].value_counts()

Значения распределены в интервале от 5000 до 150000, в основном с шагом 10000. Вероятнее всего на ресурсе, с которого получены указанные объявления, ввод пробега осуществляется путем выбора из некоторого списка (судя по всему с ограничением: минимальное значение - от 0 до 5000, максимальное - свыше 150000).  Можно отметить, что признак пробега можно отнести к категориальным - количество уникальных значений ограничено (судя по всему, на указанном ресурсе выбор пробега осуществляется только путем выбора из списка значений).

Посмотрим на распределение видов топлива.

In [ ]:
data['fuel_type'].value_counts()

Распределение по виду топлива крайне неравномерно - 98% приходятся на виды топлива petrol и gasoline. Учитывая то, что указанные слова являются синонимами, целесообразно:

* заменить эти категории на одну
* имеющиеся пропуски (учитывая их существенный объем относительного всех данных) заменить также на наиболее частую категорию - petrol

Также целесообразно категории lpg и cng объединить в одну, поскольку они относятся к категории газового вида топлива. А оставшиеся категории малого объема предложений (гибридные, электродвигатели) исключить.

In [ ]:
data.loc[data['fuel_type'] == 'gasoline', 'fuel_type'] = 'petrol'

In [ ]:
data['fuel_type'] = data['fuel_type'].fillna('petrol')

In [ ]:
data.loc[data['fuel_type'] == 'lpg', 'fuel_type'] = 'gas'
data.loc[data['fuel_type'] == 'cng', 'fuel_type'] = 'gas'

In [ ]:
data = data.query('fuel_type in (\'petrol\', \'gas\')')

In [ ]:
data['fuel_type'].value_counts()

In [ ]:
data['model'].value_counts()

In [ ]:
data['model'].isnull().sum() / data.shape[0]

Модельный ряд представлен списком из порядка 250 различных наименований. Учитывая то, что помимо пропусков в данных присутствует значение other (прочие модели), целесообразно заменить пропуски указанным значением.

In [ ]:
data['model'] = data['model'].fillna('other')

Учитывая то, что уникальных значений признака модели достаточно много (около 250), целесообразно для обучения ML-модели создать производный признак, который бы содержал наиболее распространенные модели, а остальные относил бы к категории "прочие".

Указанную операцию выполним на этапе подготовки обучающей и тестовой выборок (по статистике обучающей выборки).

Посмотрим на типы коробки передач.

In [ ]:
data['gearbox'].isnull().sum()

In [ ]:
data['gearbox'].value_counts()

Соотношение ручная/автоматическая коробка передач - 4 к 1. Учитывая относительно небольшой объем пропусков данных, нецелесообразно выделять отдельную категорию (например, unknown) - имеет смысл заменить на наиболее распространенное значение - manual.

In [ ]:
data['gearbox'] = data['gearbox'].fillna('manual')

Оценим распределение значений признака типа кузова.

In [ ]:
data['vehicle_type'].value_counts()

In [ ]:
data['vehicle_type'].isnull().sum()

Присутствуют данные с пропусками в типе кузова, при этом для многих подобных объявлений указаны марка и модель, что позволит выполнить замену, например, на наиболее распространенный тип кузова для модели.

Аналогично замене 0 признака мощности, чтобы не переобучить модель указанную операцию целесообразно выполнить после разделения всей выборки на обучающую и тестовую (на обучающей собрать статистику и использовать ее для замены значений на обучающей и тестовой выборках).

#### Разделение данных на обучающую и тестовую выборку. Заполнение пропусков на основе статистики в обучающей выборке.

Выполним стратифицированное разделение данных на обучающую и тестовую выборку. В качестве разделяющего признака используем добавленный выше признак brand_type, позволяющий отделить производителей, занимающих более 1% рынка от производителей с небольшой долей.

In [ ]:
df_train, df_test = train_test_split(
        data, test_size=0.2, random_state=random_magic, stratify=data['brand_type']
    )

Подготовим статистические данные для замены пропусков/аномалий в признаках на основе имеющихся в обучающей выборке данных - значениях мощности и типа кузова для конкретных производителей и моделей автомобилей.

In [ ]:
df_train = df_train.copy()
df_test = df_test.copy()

Определим медианные мощности по производителям и маркам автомобилей на основе данных обучающей выборки.

In [ ]:
power_median = df_train.query('power > 0 and brand_type == \'other\'')['power'].median()
power_median

In [ ]:
df_model_power = (
    df_train
    .query('power > 0')
    .groupby(['brand', 'model'])['power']
    .median()
    .reset_index()
)

df_model_power

In [ ]:
def find_power_median_value(brand, model):
    """
    функция возвращает медианное значение мощности для входных параметров 
    brand - марка автомобиля
    model - модель автомобиля
    при отсутствии в таблице медианных значений указанной модели - возврат медианного значения мощности 
    по всей таблице (без учета пустых значений)
    
    """
    power_group = df_model_power.query('brand == @brand and model == @model')['power']

    if power_group.empty:
        return power_median
    else:
        return power_group.values[0]

Выполним замену в обучающей и тестовой выборках на основе сформированной статистики.

In [ ]:
df_train.loc[df_train['power'] == 0,  'power'] = df_train.query('power == 0').apply(
    lambda row: find_power_median_value(row['brand'], row['model']),
    axis=1
).astype('int')

In [ ]:
df_test.loc[df_test['power'] == 0, 'power'] = df_test.query('power == 0').apply(
    lambda row: find_power_median_value(row['brand'], row['model']),
    axis=1
).astype('int')

Определим наиболее часто встречаемые типы кузовов по производителям и моделям.

In [ ]:
vehicle_type_top = df_train['vehicle_type'].mode()[0]
vehicle_type_top

In [ ]:
df_model_vehicle_type = (
    df_train.groupby(['brand', 'model'])['vehicle_type'].
    agg(lambda x:x.value_counts().index[0])
    .reset_index()
)

df_model_vehicle_type

In [ ]:
def find_vehicle_type_top_value(brand, model):
    """
    функция возвращает медианное значение мощности для входных параметров 
    brand - марка автомобиля
    model - модель автомобиля
    при отсутствии в таблице медианных значений указанной модели - возврат медианного значения мощности 
    по всей таблице (без учета пустых значений)
    
    """
    vehicle_type_group = df_model_vehicle_type.query('brand == @brand and model == @model')['vehicle_type']

    if vehicle_type_group.empty:
        return vehicle_type_top
    else:
        return vehicle_type_group.values[0]

Аналогичным образом заполняем пропуски в обучающей и тестовой выборках

In [ ]:
df_train.loc[df_train['vehicle_type'].isnull(),  'vehicle_type'] = (
    df_train.query('vehicle_type.isnull()', engine='python').apply(
    lambda row: find_vehicle_type_top_value(row['brand'], row['model']),
    axis=1)
)

In [ ]:
df_test.loc[df_test['vehicle_type'].isnull(),  'vehicle_type'] = (
    df_test.query('vehicle_type.isnull()', engine='python').apply(
    lambda row: find_vehicle_type_top_value(row['brand'], row['model']),
    axis=1)
)

Добавим производный обучающий признак с целью выделить наиболее распространенные модели.

In [ ]:
df_model_top = df_train.pivot_table(
    index='model', 
    values='brand', 
    aggfunc=['count']
).reset_index()

df_model_top.columns = ['model_top', 'count']
df_model_top = df_model_top.query('count > 1000')

In [ ]:
df_train['model_type'] = df_train.merge(
    df_model_top, left_on='model', right_on ='model_top', how='left'
)['model_top']

df_train['model_type'] = df_train['model_type'].fillna('other')

In [ ]:
df_test['model_type'] = df_test.merge(
    df_model_top, left_on='model', right_on ='model_top', how='left'
)['model_top']

df_test['model_type'] = df_test['model_type'].fillna('other')

Проверим общую информацию о признаках обучающей и тестовой выборок.

In [ ]:
df_train.info()

In [ ]:
df_test.info()

Пропуски заполнены, типы данных в обеих выборках совпадают. Можно переходить к этапу построения и обучения моделей.

### Выводы:

В таблице порядка 355 тысяч записей об объявлениях о продаже автомобилей. Целевым признаком является цена автомобиля (Price), указанная в евро.

Состав атрибутов совпадает с полученным описанием. Для удобства работы выполнено приведение названий атрибутов из 'верблюжьего' стиля в 'змеиный'.

Далее проведена оценка распределения значений признаков с целью выявления пропусков/аномалий в данных, которые целесообразно исключить или заменить.

По результатам оценки признаков, относящихся к самим объявлениям (дата создания, дата выгрузки), исключены аномалии - объявления с большим периодом экспозиции, которые вероятно содержат недостоверные ценовые характеристики. Также исключены аномальные значения года регистрации автомобилей, а также значения-выбросы, не дающие представления о текущем состоянии рынка автомобилей.

По результатам оценки числовых и категориальных признаков также устранены отдельные аномалии и выполнены замены пропусков значений.

После выполнения указанных операций в таблице осталось порядка 330 тясяч записей. Указанная выборка разделена на обучающую и тестовую (с пропорциональным разбиением по производителю). На основе статистики обучающей выборки выполнена замена пропусков значений мощности, типа кузова и модели автомобилей.

Данные подготовлены для этапа обучения моделей.

## Обучение моделей

Построим различные модели машинного обучения.

Определим наборы обучающих и целевого признаков, выделим категориальные и числовые (подлежащие масштабированию) признаки

In [ ]:
features = ['vehicle_type',
            'registration_year',
            'gearbox',
            'power',
            'model_type',
            'kilometer',
            'registration_month',
            'fuel_type',
            'brand',
            'not_repaired']

features_category = ['vehicle_type', 'gearbox', 'model_type', 'kilometer', 
                     'registration_month', 'fuel_type', 'brand', 'not_repaired']

features_scale = ['registration_year', 'power']

target = ['price']

In [ ]:
X_train = df_train[features]
Y_train = df_train[target]

X_test = df_test[features]
Y_test = df_test[target]

Создадим функцию расчета метрики RMSE

In [ ]:
def rmse(Y_true, Y_predicted):
    return mean_squared_error(Y_true, Y_predicted) ** 0.5

In [ ]:
scoring = make_scorer(rmse, greater_is_better=False)

Определим функции вывода значений метрики по результатам обучения и проверки моделей на тестовой выборке.

In [ ]:
def print_scores(grid_search, params):
    """
    функция печати сводной информации о параметрах модели
    
    """
    print ('Наилучшее значение метрики RMSE: %0.3f' % np.abs(grid_search.best_score_))
    print ('Наилучшие параметры:')
    best_parameters = grid_search.best_estimator_.get_params()
    
    for param_name in sorted(params.keys()):
        print ('\t%s: %r' % (param_name, best_parameters[param_name]))

In [ ]:
def print_scores_on_test(grid_search, params, X_test, Y_test):
    print ('Параметры модели:')
    best_parameters = grid_search.best_estimator_.get_params()
    grid_parameters = {}
    
    for param_name in sorted(params.keys()):
        grid_parameters[param_name] = best_parameters[param_name]
        print ('\t%s: %r' % (param_name, best_parameters[param_name]))
    
    score_value = np.abs(grid_search.score(X_test, Y_test))
    print('Значение метрики RMSE на тестовых данных:', score_value)
    
    return score_value, grid_parameters

Для последующего сравнения моделей объявим переменную для хранения статистической информации по качеству и времени обучения моделей.

In [ ]:
ml_data = []

#### Dummy-модель


Для последующей оценки полученных результатов проверки моделй на тестовых данных рассмотрим несколько dummy-моделей, выполняющий предсказание по одному из простых принципов:

* медианное значение цены
* среднее значение по бренду
* среднее значение по модели

In [ ]:
dummy = DummyRegressor(strategy="median")
dummy.fit(X_train, Y_train)

print(
    'Значение dummy метрики RMSE на тестовых данных:', 
    rmse(Y_test.values.ravel(), dummy.predict(X_test)) 
)

In [ ]:
df_brand_median_price = df_train.pivot_table(
    index='brand', 
    values='price', 
    aggfunc=['mean']
).reset_index()

df_brand_median_price.columns = ['brand', 'price']

In [ ]:
predictions = df_test.merge(
    df_brand_median_price, left_on='brand', right_on ='brand', how='inner'
)['price_y']

print(
    'Значение dummy метрики RMSE на тестовых данных:', 
    rmse(Y_test.values.ravel(), predictions) 
)

In [ ]:
df_model_median_price = df_train.pivot_table(
    index='model_type', 
    values='price', 
    aggfunc=['mean']
).reset_index()

df_model_median_price.columns = ['model_type', 'price']

In [ ]:
predictions = df_test.merge(
    df_model_median_price, left_on='model_type', right_on ='model_type', how='inner'
)['price_y']

print(
    'Значение dummy метрики RMSE на тестовых данных:', 
    rmse(Y_test.values.ravel(), predictions) 
)

Базовое значение ошибки для построения моделей машинного обучения получено.

In [ ]:
ml_data.append(['dummy', 0, 0, 4499, {}])

In [ ]:
cv_folds = 5
simple_param = True

#### Линейная регрессия

В качестве шагов преобразования данных определим масштабирование и кодирование с помощью OrdinalEncoder (целочисленный код категории). Указанный способ кодирования категориальных признаков ближе к применяемым к CatBoost и LightGBM численным способам кодирования.

In [ ]:
transform_steps = [
    (
        'qtl', ColumnTransformer(
            remainder='passthrough',
            transformers=[  
                ('std', StandardScaler(copy=False, with_mean=True, with_std=True), features_scale),
                ('cat', OrdinalEncoder(), features_category)
            ])
    ),
]

In [ ]:
model_lr = LinearRegression()

params = {
    'model__fit_intercept': [True, False],
    'model__normalize': [True, False]
}

In [ ]:
def get_param_count(params):
    return np.prod([len(value) for key, value in params.items()])

In [ ]:
grid_search = GridSearchCV(
        Pipeline(transform_steps + [('model', model_lr)]), 
        param_grid=params, 
        cv=cv_folds, 
        scoring=scoring,
    ) 

In [ ]:
%%time
start_time = datetime.datetime.now()

grid_search.fit(X_train, Y_train.values.ravel())

learning_time = (datetime.datetime.now()-start_time).seconds

print_scores(grid_search, params)

In [ ]:
rmse_score, grid_parameters = print_scores_on_test(grid_search, params, X_test, Y_test.values.ravel())

ml_data.append(['Линейная регрессия', learning_time, learning_time / get_param_count(params), rmse_score, grid_parameters])

С помощью линейной регрессии удалось достичь значения метрики RMSE = 2978, что  в 1.5 раза лучше, чем dummy-модель

#### Случайный лес

Рассмотрим модель случайного леса. В качестве диапазона перебираемых гиперпараметров укажем глубину дерева и количество деревьев в лесу.

In [ ]:
model_rfr = RandomForestRegressor(random_state=random_magic)

params = {
    'model__n_estimators': [10, 20, 30],
    'model__max_depth': [2, 3, 5],
}

if simple_param:
    params = {
        'model__n_estimators': [30],
        'model__max_depth': [5],
    }

In [ ]:
grid_search = GridSearchCV(
        Pipeline(transform_steps + [('model', model_rfr)]), 
        param_grid=params, 
        cv=cv_folds, 
        scoring=scoring,
    )

In [ ]:
%%time
start_time = datetime.datetime.now()

grid_search.fit(X_train, Y_train.values.ravel())

learning_time = (datetime.datetime.now()-start_time).seconds

print_scores(grid_search, params)

In [ ]:
rmse_score, grid_parameters = print_scores_on_test(grid_search, params, X_test, Y_test.values.ravel())

ml_data.append(['Случайный лес', learning_time, learning_time / get_param_count(params), rmse_score, grid_parameters])

Модель случайного леса позволила получить значения метрики RMSE = 2351, что более чем на 20% лучше модели линейной регрессии.

#### CatBoost

Рассмотрим модель градиентного бустинга CatBoostRegressor. В качестве метрики используем RMSE

In [ ]:
class RmseMetric(object):
    def get_final_error(self, error, weight):
        return np.sqrt(error / (weight + 1e-38))

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += w * ((approx[i] - target[i])**2)

        return error_sum, weight_sum

In [ ]:
model_cbr = CatBoostRegressor(
    verbose=50,
    #eval_metric='RMSE',
    eval_metric=RmseMetric(),
    cat_features=features_category,
    task_type="CPU",
    iterations=1000,  
)

params = {
    'learning_rate': [0.03, 0.05, 0.15],
    'depth': [5, 7, 10],
}

if simple_param:
    params = {
        'learning_rate': [0.15],
        'depth': [7],
    }

In [ ]:
%%time

start_time = datetime.datetime.now()

model_cbr.fit(
    df_train[features], df_train[target],
    plot=True
)

learning_time = (datetime.datetime.now()-start_time).seconds

In [ ]:
rmse_score = rmse(Y_test.values.ravel(), model_cbr.predict(X_test))

print('Значение метрики RMSE на тестовых данных:', rmse_score)

ml_data.append(['CatBoost', learning_time, learning_time / get_param_count(params), rmse_score, {}])

Модель CatBoostRegressor библиотеки CatBoost позволила получить значение метрики RMSE на тестовых данных ~ 1633, что ниже рассмотренных выше моделей.

#### LightGBM

Рассмотрим модель градиентного бустинга LGBMRegressor.

Поскольку для модели LGBMRegressor категориальные признаки требуются в формате категориальных типов данных - выполним склейку обучающей и тестовой выборки, преобразование категориальных типов и последующее обратное разделение (по имеющимся индексам).

In [ ]:
df_lgbm = df_train.append(df_test)

In [ ]:
for c in df_lgbm.columns:
    col_type = df_lgbm[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        df_lgbm[c] = df_lgbm[c].astype('category')

In [ ]:
df_train_lgbm = df_lgbm.loc[df_train.index]
df_test_lgbm = df_lgbm.loc[df_test.index]

In [ ]:
X_train_lgbm = df_train_lgbm[features]
Y_train_lgbm = df_train_lgbm[target]

X_test_lgbm = df_test_lgbm[features]
Y_test_lgbm = df_test_lgbm[target]

In [ ]:
model_lgbm = lgb.LGBMRegressor(
    random_state=random_magic,
    device="cpu"
)

params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.03, 0.05, 0.15],
}

if simple_param:
    params = {
        'max_depth': [7],
        'learning_rate': [0.15],
    }

In [ ]:
%%time

start_time = datetime.datetime.now()

grid_search = GridSearchCV(
        estimator=model_lgbm,
        param_grid=params, 
        cv=cv_folds, 
        scoring=scoring,
        n_jobs=-1, 
        verbose=5
    )
fitted_model = grid_search.fit(X_train_lgbm, Y_train_lgbm)

learning_time = (datetime.datetime.now()-start_time).seconds

print_scores(grid_search, params)

In [ ]:
rmse_score, grid_parameters = print_scores_on_test(grid_search, params, X_test_lgbm, Y_test_lgbm.values.ravel())

ml_data.append(['LightGBM', learning_time, learning_time / get_param_count(params), rmse_score, grid_parameters])

Модель LGBMRegressor библиотеки LightGBM позволила получить значение метрики RMSE на тестовых данных ~ 1669, что немного хуже CatBoost.

## Анализ моделей

In [ ]:
df_ml_compare = pd.DataFrame(ml_data, columns=['model', 'learning_time', 'learning_time_per_param', 'rmse', 'params'])
df_ml_compare

Линейная регрессия позволила получить значение метрики равное ~ 2978, случайный лес ~ 2351, обе модели градиентного бустинга ~ 1633-1669. Все из перечисленных моделей значительно улучшили предсказание dummy-моделей (~ 4500), при этом качество моделей градиентного бустинга почти в 2 раза лучше линейной регрессии и на 25% лучше случайного леса.

Время обучения модели градиентного бустинга LGBMRegressor незначительно превышает время обучения линейной регрессии. 

Качество моделей градиентного бустинга на 25% выше случайного леса и существенного превосходят оценки dummy-модели. Время обучения CatBoost относительно LightGBM на порядок хуже.

### Выводы:

В рамках исследования проведено построение моделей машинного обучения с целью предсказания на основе технических характеристик цены автомобилей.

Проведена обработка данных (порядка 355 тысяч записей об объявлениях о продаже автомобилей):

* проведена оценка распределения значений признаков с целью выявления пропусков/аномалий в данных, которые целесообразно исключить или заменить.
* по результатам оценки признаков, относящихся к самим объявлениям (дата создания, дата выгрузки), исключены аномалии - объявления с большим периодом экспозиции, которые вероятно содержат недостоверные ценовые характеристики. Также исключены аномальные значения года регистрации автомобилей, а также значения-выбросы, не дающие представления о текущем состоянии рынка автомобилей.
* по результатам оценки числовых и категориальных признаков также устранены отдельные аномалии и выполнены замены пропусков значений.

После выполнения указанных операций в таблице осталось порядка 330 тысяч записей. Указанная выборка разделена на обучающую и тестовую (с пропорциональным разбиением по производителю). На основе статистики обучающей выборки выполнена замена пропусков значений мощности, типа кузова и модели автомобилей.

Проведено обучение 4 различных типов моделей:

* линейная регрессия
* случайный лес
* градиентный бустинг CatBoost
* градиентный бустинг LightGBM

Линейная регрессия позволила получить значение метрики равное ~ 2978, случайный лес ~ 2351, обе модели градиентного бустинга ~ 1633-1669. Все из перечисленных моделей значительно улучшили предсказание dummy-моделей (~ 4500), при этом качество моделей градиентного бустинга почти в 2 раза лучше линейной регрессии и на 25% лучше случайного леса.

Время обучения модели градиентного бустинга LGBMRegressor незначительно превышает время обучения линейной регрессии. 

Качество моделей градиентного бустинга на 25% выше случайного леса и существенного превосходят оценки dummy-модели. Время обучения CatBoost относительно LightGBM на порядок хуже.